In [ ]:
# 데이터 구조
import pandas as pd
# 데이터 시각화
import matplotlib.pyplot as plt ; import matplotlib
# 나무 구조 시각화
from sklearn.tree import plot_tree
# 데이터 분할: Train, Test
from sklearn.model_selection import train_test_split
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
# 최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV
# 모델 성능 평가
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
# 한글 폰트 설정
matplotlib.rc("font", family = "Malgun Gothic")
# 음수 표시 설정
matplotlib.rc("axes", unicode_minus = False)

In [ ]:
# 데이터 불러오기
df_raw = pd.read_csv("../data/bank.csv")
df_raw.head()

In [ ]:
# 결측치 확인
df_raw.isnull().sum(axis = 0)
# JOB 변수의 결측치는 "Other"로 대체
df_raw["JOB"].fillna("Other", inplace = True)
# 숫자형 변수의 결측치는 해당 변수의 평균값으로 대체
df_raw.fillna(df_raw.mean(), inplace=True)

In [ ]:
# get_dummies: 데이터의 문자형 변수에 대한 더미변수 생성
df_raw_dummy = pd.get_dummies(df_raw)
# 더미변수 생성된 데이터의 상위 5개 row를 확인
df_raw_dummy.head()

In [ ]:
# 학습용/평가용 데이터 분리
df_train, df_test = train_test_split(df_raw_dummy, test_size = 0.3, random_state = 1234)
print("학습용 데이터의 크기 : {}".format(df_train.shape))
print("평가용 데이터의 크기 : {}".format(df_test.shape))

In [ ]:
# 설명변수와 목표변수를 분리
df_train_y = df_train["BAD"]
df_train_x = df_train.drop("BAD", axis = 1, inplace = False)
df_test_y = df_test["BAD"]
df_test_x = df_test.drop("BAD", axis = 1, inplace = False)

In [ ]:
tree_uncustomized = DecisionTreeClassifier(random_state=1234)
tree_uncustomized.fit(df_train_x, df_train_y)
# 학습용 데이터 정확도
print("학습용 데이터의 정확도 : {:.3f}".format(tree_uncustomized.score(df_train_x, df_train_y)))
# 평가용 데이터 정확도
print("평가용 데이터의 정확도 : {:.3f}".format(tree_uncustomized.score(df_test_x, df_test_y)))

In [ ]:
estimator = DecisionTreeClassifier()
# 구하고자 하는 parameter와 범위
param_grid = {"max_depth": [4, 5, 6, 7], "min_samples_split": [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
"min_samples_leaf": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
# 정확도가 높은 최적 parameter 찾기
grid_dt = GridSearchCV(estimator, param_grid, scoring="accuracy", n_jobs = -1)
grid_dt.fit(df_train_x, df_train_y)
print("best estimator model: \n{}".format(grid_dt.best_estimator_))
print("\nbest parameter: \n{}".format(grid_dt.best_params_))
print("\nbest score: \n{}".format(grid_dt.best_score_.round(3)))

In [ ]:
tree_final = DecisionTreeClassifier(max_depth = 6, min_samples_split = 8, min_samples_leaf =1,
random_state=1234)
tree_final.fit(df_train_x, df_train_y)
# 학습용 데이터 정확도
print("학습용 데이터의 정확도: {:.3f}".format(tree_final.score(df_train_x, df_train_y)))
# 평가용 데이터 정확도
print("평가용 데이터의 정확도: {:.3f}".format(tree_final.score(df_test_x, df_test_y)))

In [ ]:
# 변수명 저장
v_feature_name = df_train_x.columns
plt.figure(figsize = (10, 7))
plot_tree(tree_final,
feature_names = v_feature_name,
filled = True);

In [ ]:
# tree.feature_importances로 설명변수 중요도 확인 및 테이블로 저장
df_importance = pd.DataFrame()
df_importance["Feature"] = df_train_x.columns
df_importance["Importance"] = tree_final.feature_importances_
# df_feature_importance의 테이블을 중요도별로 정렬
df_importance.sort_values("Importance", ascending=False, inplace = True)
df_importance.round(3)

In [ ]:
# 설명변수 중요도 그래프
df_importance.sort_values("Importance", ascending=True, inplace = True)
coordinates = range(len(df_importance))
plt.barh(y = coordinates, width = df_importance["Importance"])
plt.yticks(coordinates, df_importance["Feature"])
plt.xlabel("변수 중요도")
plt.ylabel("변수")